<a href="https://colab.research.google.com/github/Sathira443/ML-Lab-1/blob/main/Lab1_label1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read the data from csv files and import libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/valid.csv")

Separate features and labels

In [ ]:
label_columns = ['label_1', 'label_2', 'label_3', 'label_4']
df_train_X = df_train.drop(label_columns, axis=1)
df_train_y = df_train['label_1']
df_test_X = df_test.drop(label_columns, axis=1)
df_test_y = df_test['label_1']

Standardize the data

In [ ]:
scaler = StandardScaler()
df_train_X_scaled = scaler.fit_transform(df_train_X)
df_test_X_scaled = scaler.transform(df_test_X)

Train initial model using RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(df_train_X_scaled, df_train_y)
y_pred_before = model.predict(df_test_X_scaled)

Apply PCA

In [ ]:
pca = PCA(0.95)
pca.fit(df_train_X_scaled)
pca_train_X = pca.transform(df_train_X_scaled)
pca_test_X = pca.transform(df_test_X_scaled)

Train model after PCA

In [ ]:
model_after = RandomForestClassifier()
model_after.fit(pca_train_X, df_train_y)
y_pred_after = model_after.predict(pca_test_X)

Feature importance and selection

In [ ]:
importance = model_after.feature_importances_
selected_columns = importance >= 0.01
reduced_pca_train_X = pca_train_X[:, selected_columns]
reduced_pca_test_X = pca_test_X[:, selected_columns]

Train final model

In [ ]:
model_final = RandomForestClassifier()

model_final.fit(reduced_pca_train_X, df_train_y)
y_pred_final = model_final.predict(reduced_pca_test_X)

Create a DataFrame for results

In [ ]:
rows, columns = reduced_pca_train_X.shape

feature_names = ["feature_" + str(i) for i in range(1, columns + 1)]

data_frame = pd.DataFrame(reduced_pca_test_X, columns=feature_names)

new_columns = {
    'No. of features': columns,
    'Predicted labels after' : y_pred_final,
    'Predicted labels before' : y_pred_before
}


Concatenate feature data and results

In [ ]:
results_df = pd.DataFrame(new_columns)

reduced_pca_test_X_df = pd.DataFrame(reduced_pca_test_X, columns=feature_names)
final_df = pd.concat([results_df, reduced_pca_test_X_df], axis=1)

csv_file_path = '190359P_label_1.csv'
final_df.to_csv(csv_file_path, index=False)